In [ ]:
#| default_exp matching

# Matcher
The Matcher abstract class defines the interface for implementations that match response units with target items. Example concrete implementations of this class can utilize methods such as keyword matching, similarity measures, or machine learning algorithms.

In [ ]:
#| export
from abc import ABC, abstractmethod
from typing import List, Union, Dict
import numpy as np

class Matcher(ABC):
    
    """
    Abstract base class for implementing response unit matching 
    strategies. To create a custom matcher, inherit from this 
    class and override the match method.
    """ 

    @abstractmethod
    def __call__(self, 
               response_units: Union[List[str], List[Dict[str, object]]], 
               target_items: Union[List[str], List[Dict[str, object]]],
               response_context: str, target_context: str) -> np.ndarray:
        """
        Matches the response_units to target_items based on a specific strategy.

        Args:
            response_units (Union[List[str], List[Dict[str, object]]]): List of response units. 
                Each response unit can be a string or a dictionary (when both text and spans are available).

            target_items (Union[List[str], List[Dict[str, object]]]): List of target items. 
                Each response unit can be a string or a dictionary (when both text and spans are available).

            response_context (str): The context of the response units.
            target_context (str): The context of the target items.

        Returns:
            np.ndarray: A boolean matrix of shape (len(target_items), len(response_units)). 
                Each element in the matrix indicates whether the corresponding
                response unit matches the corresponding target item.
        """

        pass

## Maximum Score Matcher

In [ ]:
#| export
import numpy as np
from response_sequencer.scoring import Scorer
from typing import Optional
from response_sequencer.filtering import MatchFilter

class MaximumScoreMatcher(Matcher):
    """
    Concrete Matcher class that implements a maximum score approach
    for matching response units with target items. Utilizes a provided Scorer
    to compute scores and a Thresholder for filtering out low-quality matches.
    """

    def __init__(self, scorer: Scorer, match_filter: Optional[MatchFilter] = None):
        """
        Initializes the MaximumScoreMatcher with a specified scorer and thresholder.

        Args:
            scorer (Scorer): An initialized Scorer object.
            match_filter Optional[MatchFilter]: Optional initialized MatchFilter object.
        """
        self.scorer = scorer
        self.match_filter = match_filter

    def __call__(self, 
               response_units: Union[List[str], List[Dict[str, object]]], 
               target_items: Union[List[str], List[Dict[str, object]]],
               response_context: str = '', target_context: str = '') -> np.ndarray:
        """
        Matches the response_units to target_items based on a specific strategy.

        Args:
            response_units (Union[List[str], List[Dict[str, object]]]): List of response units. 
                Each response unit can be a string or a dictionary (when both text and spans are available).

            target_items (Union[List[str], List[Dict[str, object]]]): List of target items. 
                Each response unit can be a string or a dictionary (when both text and spans are available).

            response_context (str): The context of the response units.
            target_context (str): The context of the target items.

        Returns:
            np.ndarray: A boolean matrix of shape (len(target_items), len(response_units)). 
                Each element in the matrix indicates whether the corresponding
                response unit matches the corresponding target item.
        """

        scores_matrix = self.scorer(response_units, target_items, response_context, target_context)
        if self.match_filter is not None:
            scores_matrix = self.match_filter(response_units, target_items, scores_matrix)

        max_indices = np.argmax(scores_matrix, axis=1)
        max_values = scores_matrix[np.arange(len(target_items)), max_indices]

        match_matrix = np.zeros((len(target_items), len(response_units)), dtype=bool)
        match_matrix[np.arange(len(target_items)), max_indices] = (max_values != -np.inf)
        return match_matrix


## Optimal Single Assignment Matcher

In [ ]:
#| export
import numpy as np
from scipy.optimize import linear_sum_assignment
from response_sequencer.scoring import Scorer
from response_sequencer.filtering import MatchFilter

class OptimalSingleAssignmentMatcher(Matcher):
    """
    Concrete Matcher class that implements an optimal assignment approach
    for matching response units with target items. Utilizes a provided Scorer
    to compute similarity scores and a Thresholder for filtering out low-quality matches.
    """

    def __init__(self, scorer: Scorer, match_filter: Optional[MatchFilter] = None):
        """
        Initializes the MaximumScoreMatcher with a specified scorer and thresholder.

        Args:
            scorer (Scorer): An initialized Scorer object.
            match_filter Optional[MatchFilter]: Optional initialized MatchFilter object.
        """
        self.scorer = scorer
        self.match_filter = match_filter

    def __call__(self, 
               response_units: Union[List[str], List[Dict[str, object]]], 
               target_items: Union[List[str], List[Dict[str, object]]],
               response_context: str = '', target_context: str = '') -> np.ndarray:
        """
        Matches the response_units to target_items based on a specific strategy.

        Args:
            response_units (Union[List[str], List[Dict[str, object]]]): List of response units. 
                Each response unit can be a string or a dictionary (when both text and spans are available).

            target_items (Union[List[str], List[Dict[str, object]]]): List of target items. 
                Each response unit can be a string or a dictionary (when both text and spans are available).

            response_context (str): The context of the response units.
            target_context (str): The context of the target items.

        Returns:
            np.ndarray: A boolean matrix of shape (len(target_items), len(response_units)). 
                Each element in the matrix indicates whether the corresponding
                response unit matches the corresponding target item.
        """

        scores_matrix = self.scorer(response_units, target_items, response_context, target_context)
        if self.match_filter is not None:
            scores_matrix = self.match_filter(response_units, target_items, scores_matrix)

        target_indices, response_indices = linear_sum_assignment(-scores_matrix)

        match_matrix = np.zeros((len(target_items), len(response_units)), dtype=bool)
        valid_indices = scores_matrix[target_indices, response_indices] != -np.inf
        match_matrix[target_indices[valid_indices], response_indices[valid_indices]] = True

        return match_matrix

## Maximum Entailment Matcher (Currently Out of Date as of 3/31)
Given two sentences, are these contradicting each other, entailing one the other or are these netural? Cross encoder models provided by `sentence_transformers` were trained on the SNLI and MultiNLI datasets. They produce a value 0...1 for each of the `['contradiction', 'entailment', 'neutral']` classes. First input is premise, second input is hypothesis (the sentence that is checked if it is entailed by the premise).

Here we demo matching by an entailment or NLI cross encoder. Response units are matched to a target unit if the target unit is found to be entailed by the response unit. This helps identify responses that might differ in meaning from the target unit, but nonetheless contain its meaning. However, since coherent text naturally includes sentences that are entailed by other sentences, this approach could lead to spurious matches.

In [ ]:
from sentence_transformers import CrossEncoder
from itertools import product


class MaximumEntailmentMatcher(Matcher):
    def __init__(self, model_name: str):
        """
        Initialize the MaximumEntailmentMatcher with a specified model.

        :param model_name: The name of the CrossEncoder model to use.
        """
        self.model = CrossEncoder(model_name)

    def match(self, response_units: list, target_items: list) -> list:
        """
        Match response units with target items using the CrossEncoder model
        based on maximum entailment scores.

        Args:
            response_units (List[str]): List of response items.
            target_items (List[str]): List of target items.

        Returns:
            List[str]: For each response unit, a string containing the matched target item. Empty strings are used for unmatched responses
        """
        pair_scores = self.model.predict(list(product(response_units, target_items)))
        entailment_scores = []
        for pair_score in pair_scores:
            if np.argmax(pair_score) == 1:
                entailment_scores.append(pair_score[1])
            else:
                entailment_scores.append(0.0)
                
        scores_matrix = [entailment_scores[i:i + len(target_items)] for i in range(0, len(entailment_scores), len(target_items))]

        matched_items = ["" for _ in range(len(response_units))]
        for response_index, row in enumerate(scores_matrix):
            max_score_index = row.index(max(row))
            if max(row) > 0:
                matched_items[response_index] = target_items[max_score_index]

        return matched_items

## Mutual Entailment Matcher
A stronger test of equivalence between two sentences is mutual entailment. This is a two-way test, and if both directions are entailed, we can be more confident that the two sentences are equivalent.

In [ ]:
#| export
class MutualEntailmentMatcher(Matcher):
    def __init__(self, model_name: str):
        """
        Initialize the MutualEntailmentMatcher with a specified model.

        :param model_name: The name of the CrossEncoder model to use.
        """
        self.model = CrossEncoder(model_name)

    def match(self, response_units: list, target_items: list) -> list:
        """
        Match response units with target items using the CrossEncoder model
        based on mutual entailment.

        Args:
            response_units (List[str]): List of response items.
            target_items (List[str]): List of target items.

        Returns:
            List[str]: For each response unit, a string containing the matched target item. Empty strings are used for unmatched responses
        """
        forward_pair_scores = self.model.predict(list(product(response_units, target_items)))
        backward_pair_scores = self.model.predict(list(product(target_items, response_units)))

        forward_entailment_scores = []
        for pair_score in forward_pair_scores:
            if np.argmax(pair_score) == 1:
                forward_entailment_scores.append(pair_score[1])
            else:
                forward_entailment_scores.append(0.0)
        
        backward_entailment_scores = []
        for pair_score in backward_pair_scores:
            if np.argmax(pair_score) == 1:
                backward_entailment_scores.append(pair_score[1])
            else:
                backward_entailment_scores.append(0.0)

        minimum_entailment_scores = [min(forward_entailment_scores[i], backward_entailment_scores[i])
                                        for i in range(len(forward_entailment_scores))]

        forward_scores_matrix = [forward_entailment_scores[i:i + len(target_items)] for i in range(0, len(forward_entailment_scores), len(target_items))]
        minimum_scores_matrix = [minimum_entailment_scores[i:i + len(target_items)] for i in range(0, len(minimum_entailment_scores), len(target_items))]

        matched_items = ["" for _ in range(len(response_units))]
        for row_index, row in enumerate(forward_scores_matrix):
            max_score_index = row.index(max(row))
            if minimum_scores_matrix[row_index][max_score_index] > 0:
                matched_items[row_index] = target_items[max_score_index]

        return matched_items

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()